In [214]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import pathlib

import tensorflow as tf
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Data in

In [215]:
os.listdir('./kaggle')

['input']

In [216]:
os.listdir('./kaggle/input/campqmind-hackathon-2022')

['sample_submission.csv', 'test', 'train']

In [217]:
# Since our data is comprised of images, we have to load each image from each directory and store them into arrays
# 136, 102

# This function returns all the filenames in the chosen directory
def get_files(directory):
    files = []
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files

train = []
labels = []

# Alternate:
# train = np.zeros(shape(num_of_valid_images, their dimensions (102x136), 3: x, y, rgb))
# labels = np.zeros(shape(num_of valid_labels, 1))
for file in get_files('./kaggle/input/campqmind-hackathon-2022/train/'):
    img = cv2.imread(file)
    img_array = np.asarray(img)
    if img_array.shape != (102, 136, 3):
        continue
    train.append(img_array)
    category = file.split('/')[-2]
    if category == "Shoe":
        labels.append(0)
    elif category == "Boot":
        labels.append(1)
    else:
        labels.append(2)

In [218]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(102,
                                  136,
                                  3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, 3, 3, activation='relu', input_shape = (102,136,3)),
  tf.keras.layers.Conv2D(32, 3, 3, activation='relu'),
  tf.keras.layers.MaxPool2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32, activation = 'relu'),
  tf.keras.layers.Dense(3, activation='sigmoid', name = 'output')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 34, 45, 32)        896       
                                                                 
 conv2d_44 (Conv2D)          (None, 11, 15, 32)        9248      
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 5, 7, 32)         0         
 g2D)                                                            
                                                                 
 flatten_18 (Flatten)        (None, 1120)              0         
                                                                 
 dense_19 (Dense)            (None, 32)                35872     
                                                                 
 output (Dense)              (None, 3)                 99        
                                                     

In [219]:
train_np = np.asarray(train)
labels_np = np.asarray(labels).astype('float32').reshape((-1, 1))


In [220]:
train_np.shape
labels_np.shape

(10498, 1)

In [221]:
np_train = np.array(train)

In [222]:
np_train.shape

(10498, 102, 136, 3)

In [223]:
model.fit(np_train, labels_np, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/home/khoa/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sampl

In [ ]:
import csv
import glob

dir_name = './kaggle/input/campqmind-hackathon-2022/test/'
# Get list of all files in a given directory sorted by name
# get_file get files names not in order.
list_of_files = sorted( filter( os.path.isfile,
                        glob.glob(dir_name + '*') ) )

test = []
test_labels = []
with open('solutions.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    spamreader.__next__()
    for row in spamreader:
        label = row[0].split(',')[1]
        if label == "Shoe":
            test_labels.append(0)
        elif label == "Boot":
            test_labels.append(1)
        else:
            test_labels.append(2)

        file = row[0].split(',')[0]
        img = cv2.imread('./kaggle/input/campqmind-hackathon-2022/test/' + file)
        img_array = np.asarray(img)
        test.append(img_array)

test_np=np.asarray(test)
test_labels=np.array(test_labels)

In [ ]:
model.evaluate(test_np, test_labels)

141/141 [==============================] - 0s 1ms/step - loss: 1.6777 - accuracy: 0.6467


[1.677733302116394, 0.6466666460037231]

# Creating a Submission